'''
L1 Error and Regularization vs L2 Error and Regularization
'''

In [2]:
import numpy as np
import math

In [3]:
class PolynomialRegeression:
    def __init__(deg = 1, l2_err = True, l1_err = False, l2_reg = False, l1_reg = False, gradient_desc = True, normal = False):
        self.w = []
        self.degree = deg
        self.l2_err = l2_err
        self.l1_err = l1_err
        self.l2_reg = l2_reg
        self.l1_reg = l1_reg
        self.gradient_desc = gradient_desc
        self.normal = normal
    
    def predict(X):
        return self.w.dot(X)
    
    def cost_function(X, y):
        predictions = predict(X)

        # Matrix math lets use do this without looping
        sq_error = (predictions - y)**2

        # Return average squared error among predictions
        return 1.0/(2*N) * sq_error.sum()

    def fit(X, y):
        if len(X) == 0 or len(X[0]) == 0:
            raise ValueError('X must have a shape greater than 0 columns or 0 rows') 
        
        if len(X) != len(y):
            raise ValueError('X and y must have the same number of rows')
        
        self.w = np.random.randn(len(X[0]) + 1) #+1 for bias
        
        if l2_err:
            if self.gradient_descent:
                alpha = 0.5
                tolerance = 1e-5
                reg_err = math.pow(w.dot(w), 2)
                while true:
                    for index, i in enumerate(X):
                        for idx in range(0, len(i)):
                            w[idx] = w[idx] - alpha*(w.dot(i) - y[idx])*X[index][idx] + 
                    
    

In [4]:
a = 10
print(np.mean(a))

10.0
